In [1]:
import os
import pandas as pd
data = pd.read_csv('data/data.csv', header=None)
data

,0,1,2,3
0,300000,«Наёмники блокчейна» — как ими становятся?,"['Dash', 'блокчейн', 'криптовалюта', 'управлен...",data/habr/post__300000.txt
1,300002,Динамичность должна быть безопасной,"['стартапы', 'программы', 'разработка программ...",data/habr/post__300002.txt
2,300004,Издательства выступают против блокировщика рек...,"['mozilla', 'Brave', 'блокировщики рекламы', '...",data/habr/post__300004.txt
3,300006,"Выход из зоны комфорта, как новая бизнес-модель","['бизнес-модель', 'it', 'стартап', 'интернет',...",data/habr/post__300006.txt
4,300008,«Уберизация» поможет российскому рынку рекрути...,"['mail.ru group', 'headhunter', 'российский ры...",data/habr/post__300008.txt
...,...,...,...,...
177,300390,Капитализация Kamcord превысила $100 млн после...,"['Time Warner', 'time warner cable', 'kamcord'...",data/habr/post__300390.txt
178,300392,Конференция ISDEF-2016 в Казани: почему это бы...,"['isdef', 'конференция', 'ит-бизнес', '\n ...",data/habr/post__300392.txt
179,300394,AVO и другие альтернативы роумингу,"['роуминг', 'voip', 'конкуренты', 'стартапы', ...",data/habr/post__300394.txt
180,300396,Шпаргалка в офис: ключевые метрики для развити...,"['метрики', 'развитие бизнеса', '\n ...",data/habr/post__300396.txt


## Выборка
- Заголовки включены в текста
- Запрос является одним из текстов исключённый из рассмотрения

In [2]:
from utils import read_file
select = 5
publications = [' '.join((title, read_file(path))) for title, path in zip(data[1], data[3])]
request = publications[select]
publications = publications[:select] + publications[select + 1:]
joined_publications = ' '.join(publications)

In [3]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\alexa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\alexa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Предобработки
- Удаление пробельных символов
- Удаление спец символов
- Токенезация
- Стемминг
- Удаление стандартного списка стоп-слов
- Удаление наших стоп-слов
- Удаление чисел
- Приведение к нижнему регистру

In [4]:
import re
from utils import compose
def remove_stopwords(stop_words):
    return lambda tokens: (token for token in tokens if token.lower() not in stop_words)
def tolower(tokens):
    return (token.lower() for token in tokens)
def tokenize(text):
    return word_tokenize(text, language='russian')
def stem(stemmer):
    return lambda tokens: (stemmer.stem(token) for token in tokens)
def remove_numbers(tokens):
    def is_number(string):
        try:
            float(string)
            return True
        except ValueError:
            return False
    return (token for token in tokens if not is_number(token))
def remove_spaces(text):
    return re.sub(r'\s+', ' ', text)
def remove_symbols(symbols, separator=' '):
    return lambda text: re.sub(symbols + '+', separator, text)
def remove_single(tokens):
    return (token for token in tokens if len(token) != 1)

## Специальные символы
- Арифметические операции
- Знаки препинания
- Различные скобки
- Слэши
- "Мягкий" дефис: \xad 
- А также $, #, №, @, €, •

In [5]:
special_symbols = r'''[-+\%*;:\'"()[\]^$#№@!.,?/|\\<>{}—«»€•–]'''
xad_symbol = r'''\xad'''

In [6]:
stemmer = SnowballStemmer('russian')
rus_stem = stem(stemmer)

In [7]:
default_stop_words = set((stemmer.stem(stopword.lower()) for stopword in stopwords.words('russian')))
remove_default_stopwords = remove_stopwords(default_stop_words)

In [8]:
import json
with open('stopwords.json', 'r', encoding='utf-8') as f:
    own_stop_words = set((stemmer.stem(stopword.lower()) for stopword in json.loads(f.read())))
remove_own_stopwords = remove_stopwords(own_stop_words)

In [9]:
preprocesses = [remove_spaces, remove_symbols(special_symbols), remove_symbols(xad_symbol, ''), tokenize, rus_stem, remove_default_stopwords, remove_own_stopwords, remove_single, remove_numbers, tolower]

- Препроцессим текста 
- Конкатенируем заново токены через пробел

In [10]:
preprocessed_publications = [' '.join(compose(preprocesses)(publication)) for publication in publications]
joined_preprocessed_publications = ' '.join(preprocessed_publications)

## Меры
- Обычная Евклидова [0, ∞)
- Нормализованная Евклидова \[0, 2]
- Косинусное расстояние \[0, 1]

In [11]:
import scipy as sp
def simple_eucl(v1, v2):
    delta = v1 - v2
    return sp.linalg.norm(delta.toarray())
def normal_eucl(v1, v2):
    normal_v1 = v1 / sp.linalg.norm(v1.toarray())
    normal_v2 = v2 / sp.linalg.norm(v2.toarray())
    delta = normal_v1 - normal_v2
    return sp.linalg.norm(delta.toarray())
def cosine(v1, v2):
    return sp.spatial.distance.cosine(v1.toarray(), v2.toarray())

## Векторизаторы
- Количественные веса
- TF-IDF веса

In [12]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
def make_count(src, min_df=1, max_df=1.5, *, verbose=False, request=None):
    vectorizer = CountVectorizer(min_df=min_df, max_df=max_df)
    x_train = vectorizer.fit_transform(src)
    if verbose:
        sample_count, feature_count = x_train.shape
        print(f'titles_count={sample_count}, words_count={feature_count}')
    if request:
        return vectorizer, x_train, vectorizer.transform([request])
    return vectorizer, x_train
def make_tf_idf(src, *, verbose=False, request=None, top=50):
    vectorizer = TfidfVectorizer()
    x_train = vectorizer.fit_transform(src)
    if verbose:
        idxs = np.argsort(vectorizer.idf_)[::-1]
        features = vectorizer.get_feature_names()
        print(f'titles_count={x_train.shape[0]}, words_count={x_train.shape[1]}')
        print([features[i] for i in idxs[-top:]])
        print([features[i] for i in idxs[:top]])
    if request:
        return vectorizer, x_train, vectorizer.transform([request])
    return vectorizer, x_train

In [13]:
def get_distances(dist_func, src):
    return sorted(((dist_func(x_train.getrow(idx), request_vec), text, idx) for idx, text in enumerate(src)), key=lambda x: x[0])

In [14]:
def check_results(src, count=1):
    total = [(dist, get_distances(dist, src)[:count]) for dist in distances]
    for dist_func, dist_res  in total:
        print(f'-----------{dist_func.__name__}-----------')
        for idx, res in enumerate(dist_res):
            print(f'{idx + 1}. Best publication is {res[2]} with distance = {res[0]} :\n {res[1][:100]}')
    return total

## Кластеризация
- количество кластеров 7
- считаем что кластеры будут апроксимировать хабы

In [15]:
from utils import columnize
from sklearn.cluster import KMeans
def train(n_clusters=3):
    model = KMeans(n_clusters=n_clusters, init='k-means++', max_iter=100, n_init=1)
    model.fit(x_train)
    return model
def print_model():
    print('Top terms per cluster:')
    order_centroids = model.cluster_centers_.argsort()[:, ::-1]
    terms = vectorizer.get_feature_names()
    print_list = list(np.array([[f'Cluster {i}:'] + [f' {terms[j]}' for j in order_centroids[i, :10]] for i in range(n_clusters)]).flat)
    for values in columnize(print_list, cols=n_clusters):
        print(' '.join(value.ljust(20) for value in values))

In [16]:
n_clusters = 7
request = request # если захочется поменять
preprocessed_request = ' '.join(list(compose(preprocesses)(request)))
distances = [simple_eucl, normal_eucl, cosine]

## Количественный векторизатор и не предобработанные данные:

In [17]:
vectorizer, x_train, request_vec = make_count(publications, verbose=True, request=request)
results = check_results(publications, 5)

titles_count=181, words_count=26747
-----------simple_eucl-----------
1. Best publication is 92 with distance = 38.40572873934304 :
 7 ошибок, которые совершают кандидаты в социальных сетях «Ведите социальные сети так, будто завтра в
2. Best publication is 117 with distance = 38.535697735995385 :
 6 причин вашей усталости Постоянно устаете и не знаете почему? 
Наш пост поможет вам понять в чем же
3. Best publication is 143 with distance = 38.80721582386451 :
 Видео: конкурс технологических стартапов Danske Ideer 2016 Пока я нахожусь в Дании, решил сходить на
4. Best publication is 168 with distance = 38.88444419044716 :
 Киберспорт принесёт в 2016 году почти $500 млн прибыли 

Как сообщает аудиторская компания Pricewate
5. Best publication is 138 with distance = 38.961519477556315 :
 Бен Зильберман: важно идти вперёд, пока это возможно 

Самое сложное при запуске стартапа — продолжа
-----------normal_eucl-----------
1. Best publication is 157 with distance = 0.9131337427801735 :
 Лень 

### Вывод значений в кластерах

In [18]:
model = train(n_clusters)
print_model()

Top terms per cluster:
Cluster 0:           Cluster 1:           Cluster 2:           Cluster 3:           Cluster 4:           Cluster 5:           Cluster 6:          
 на                   не                   что                  на                   на                   на                   на                 
 не                   что                  на                   для                  не                   бренда               не                 
 что                  на                   загрузки             не                   что                  трафика              что                
 для                  это                  время                что                  вы                   для                  мы                 
 по                   вы                   сайта                по                   это                  рекламы              это                
 из                   то                   сделали              бизнес               как       

## TF-IDF векторизатор и не предобработанные данные:

In [19]:
vectorizer, x_train, request_vec = make_tf_idf(publications, verbose=True, request=request)
tfidf_results = check_results(publications, 5)

titles_count=181, words_count=26747
['больше', 'быть', 'поэтому', 'со', 'этого', 'который', 'во', 'вы', 'тем', 'уже', 'когда', 'того', 'также', 'они', 'чем', 'можно', 'том', 'будет', 'он', 'есть', 'время', 'этом', 'же', 'чтобы', 'мы', 'его', 'более', 'при', 'если', 'может', 'до', 'их', 'компании', 'только', 'которые', 'все', 'то', 'или', 'так', 'но', 'это', 'от', 'за', 'как', 'из', 'не', 'для', 'по', 'что', 'на']
['ящиках', 'замечала', 'xi', 'первоисточником', 'первоисточники', 'первоисточник', 'первозванного', 'xing', 'xix', 'первичных', 'первичный', 'первичной', 'первичного', 'первичная', 'первички', 'первенца', 'первенстве', 'первоклассными', 'xlhealth', 'замечаниями', 'пенсионный', 'пенсионного', 'пенсионеры', 'пенсии', 'пенициллин', 'пенг', 'пекки', 'пекка', 'пек', 'пейте', 'пейзажем', 'пейджа', 'первоклассная', 'xg', 'замужеству', 'wpbeginner', 'заметка', 'worldphone', 'заметки', 'перевес', 'перевернуть', 'перевернула', 'перевела', 'переведенной', 'переведения', 'переварить', 'пе

In [20]:
model = train(n_clusters)
print_model()

Top terms per cluster:
Cluster 0:           Cluster 1:           Cluster 2:           Cluster 3:           Cluster 4:           Cluster 5:           Cluster 6:          
 мы                   на                   baidu                talent               на                   google               не                 
 не                   не                   chrome               unleashed            бизнес               фонд                 на                 
 вам                  что                  tidal                awards               по                   acorns               что                
 роснано              вы                   airtime              стартап              для                  на                   это                
 на                   для                  kamcord              лучший               данных               фонда                то                 
 вы                   по                   video                ричардом             не        

## Количественный векторизатор и предобработанные данные:

In [21]:
vectorizer, x_train, request_vec = make_count(preprocessed_publications, verbose=True, request=preprocessed_request)
preprocessed_results = check_results(preprocessed_publications, 5)

titles_count=181, words_count=11707
-----------simple_eucl-----------
1. Best publication is 10 with distance = 33.13608305156178 :
 интересн событ произошедш выходн понедельник коротк подборк могл пропуст выходн дни динамичн должн б
2. Best publication is 61 with distance = 33.58571124749333 :
 vlog программист программирован жизн герман привет жив работа гамбург август прошл нача вест влог де
3. Best publication is 24 with distance = 33.63034344160047 :
 добав paypal оплат услуг последн врем участ жалоб зарубежн клиент оплат услуг предлага способ оплат 
4. Best publication is 87 with distance = 33.926390907374746 :
 интересн событ произошедш выходн понедельник коротк подборк могл пропуст выходн дни большинств онлай
5. Best publication is 164 with distance = 33.97057550292606 :
 интересн событ произошедш выходн понедельник коротк подборк могл пропуст выходн дни ларр пейдж ежене
-----------normal_eucl-----------
1. Best publication is 161 with distance = 1.111057742682411 :
 тип реклам

In [22]:
model = train(n_clusters)
print_model()

Top terms per cluster:
Cluster 0:           Cluster 1:           Cluster 2:           Cluster 3:           Cluster 4:           Cluster 5:           Cluster 6:          
 дан                  компан               результат            стат                 реклам               интернет             проект             
 пользовател          бизнес               загрузк              люд                  бренд                оо                   процесс            
 приложен             работ                сайт                 текст                вид                  магазин              заказчик           
 информац             нов                  врем                 чита                 трафик               ип                   бизнес             
 покупател            клиент               сдела                блог                 брендов              товар                команд             
 бизнес               пользовател          измеря               письм                интернет  

## TF-IDF векторизатор и предобработанные данные:

In [23]:
vectorizer, x_train, request_vec = make_tf_idf(preprocessed_publications, verbose=True, request=preprocessed_request)
preprocessed_tfidf_results = check_results(preprocessed_publications, 5)

titles_count=181, words_count=11707
['развит', 'использова', 'информац', 'сред', 'задач', 'вид', 'интернет', 'сайт', 'основн', 'крупн', 'последн', 'стал', 'цел', 'человек', 'след', 'необходим', 'случа', 'количеств', 'сторон', 'дел', 'говор', 'нача', 'люб', 'дела', 'времен', 'люд', 'например', 'проект', 'должн', 'явля', 'важн', 'стат', 'результат', 'вопрос', 'рынк', 'бизнес', 'клиент', 'поэт', 'дан', 'сдела', 'пользовател', 'получ', 'решен', 'част', 'врем', 'работа', 'работ', 'возможн', 'нов', 'компан']
['100а', 'опроверг', 'оповещен', 'опоздан', 'опозна', 'опознава', 'опорн', 'броненосц', 'бронежилет', 'опрашива', 'брокерск', 'бродяг', 'опробова', 'опровержен', 'оплот', 'опрометчив', 'бродкаст', 'бриллиант', 'оптимизир', 'брикма', 'оптов', 'бридж', 'опублику', 'брендов', 'опустошен', 'опцион', 'оплошн', 'бросок', 'оракл', 'операторск', 'олигопол', 'олимпийск', 'ользовател', 'омниканальн', 'букет', 'букеровск', 'буквоед', 'буйн', 'опер', 'оперативк', 'будораж', 'будильник', 'брошен', 'б

In [24]:
model = train(n_clusters)
print_model()

Top terms per cluster:
Cluster 0:           Cluster 1:           Cluster 2:           Cluster 3:           Cluster 4:           Cluster 5:           Cluster 6:          
 люд                  пользовател          клиент               товар                цел                  бизнес               компан             
 жизн                 приложен             письм                магазин              работ                дан                  google             
 компан               реклам               продукт              сайт                 страх                объект               uber               
 стартап              покупател            компан               relap                проблем              вещ                  миллиард           
 проект               мобильн              email                клиент               проект               взаимодейств         сервис             
 работ                сайт                 проект               mrr                  задач     

## Итоговые результаты по расстоянию:
Хотя при использовании текстов нужно смотреть на сами текста или на их теги

In [25]:
matrix = pd.DataFrame(data={
    'Not Preprocessed': {func.__name__: res[0][0] for func, res in results}, 
    'Preprocessed': {func.__name__: res[0][0] for func, res in preprocessed_results},
    'Not Preprocessed TF-IDF': {func.__name__: res[0][0] for func, res in tfidf_results}, 
    'Preprocessed TF-IDF': {func.__name__: res[0][0] for func, res in preprocessed_tfidf_results},
    }).transpose()
matrix

,simple_eucl,normal_eucl,cosine
Not Preprocessed,38.405729,0.913134,0.416907
Preprocessed,33.136083,1.111058,0.617225
Not Preprocessed TF-IDF,1.272257,1.272257,0.809319
Preprocessed TF-IDF,1.225174,1.225174,0.750526
